<a href="https://colab.research.google.com/github/mjk022/Django/blob/main/examples/actor_critic_cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actor Critic Method

**Author:** [Apoorv Nandan](https://twitter.com/NandanApoorv)<br>
**Date created:** 2020/05/13<br>
**Last modified:** 2024/02/22<br>
**Description:** Implement Actor Critic Method in CartPole environment.

## Introduction

This script shows an implementation of Actor Critic method on CartPole-V0 environment.

### Actor Critic Method

As an agent takes actions and moves through an environment, it learns to map
the observed state of the environment to two possible outputs:

1. Recommended action: A probability value for each action in the action space.
   The part of the agent responsible for this output is called the **actor**.
2. Estimated rewards in the future: Sum of all rewards it expects to receive in the
   future. The part of the agent responsible for this output is the **critic**.

Agent and Critic learn to perform their tasks, such that the recommended actions
from the actor maximize the rewards.

### CartPole-V0

A pole is attached to a cart placed on a frictionless track. The agent has to apply
force to move the cart. It is rewarded for every time step the pole
remains upright. The agent, therefore, must learn to keep the pole from falling over.

### References

- [Environment documentation](https://gymnasium.farama.org/environments/classic_control/cart_pole/)
- [CartPole paper](http://www.derongliu.org/adp/adp-cdrom/Barto1983.pdf)
- [Actor Critic Method](https://hal.inria.fr/hal-00840470/document)


## Setup


In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

# Uninstall gym and install gymnasium for compatibility with newer numpy
!pip uninstall gym -y
!pip install gymnasium

import gymnasium as gym
import numpy as np
import keras
from keras import ops
from keras import layers
import tensorflow as tf

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.12/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API w

In [2]:
# Configuration parameters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
max_steps_per_episode = 10000
# Adding `render_mode='human'` will show the attempts of the agent
env = gym.make("CartPole-v0")  # Create the environment using gymnasium
env.reset(seed=seed)
eps = np.finfo(np.float32).eps.item()  # Smallest number such that 1.0 + eps != 1.0

## Implement Actor Critic network

This network learns two functions:

1. Actor: This takes as input the state of our environment and returns a
probability value for each action in its action space.
2. Critic: This takes as input the state of our environment and returns
an estimate of total rewards in the future.

In our implementation, they share the initial layer.


In [3]:
num_inputs = 4
num_actions = 2
num_hidden = 128

inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden, activation="relu")(inputs)
action = layers.Dense(num_actions, activation="softmax")(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs=inputs, outputs=[action, critic])


## Train


In [5]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
huber_loss = keras.losses.Huber()
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0

In [8]:
while True:  # Run until solved
    _state_raw, _ = env.reset(seed=seed)
    # Ensure state is a (num_inputs,) numpy array, even if the environment returns a scalar
    if not isinstance(_state_raw, np.ndarray):
        # If it's a scalar, create a (4,) array with the scalar as the first element
        state = np.array([float(_state_raw)] + [0.0] * (num_inputs - 1), dtype=np.float32)
    else:
        # If it's already a numpy array, ensure it's (4,)
        if _state_raw.shape != (num_inputs,):
            # Handle cases where array is not (4,). Pad with zeros or truncate.
            temp_state = np.zeros(num_inputs, dtype=np.float32)
            temp_state[:min(_state_raw.shape[0], num_inputs)] = _state_raw[:min(_state_raw.shape[0], num_inputs)]
            state = temp_state
        else:
            state = _state_raw.astype(np.float32)

    episode_reward = 0
    with tf.GradientTape() as tape:
        for timestep in range(1, max_steps_per_episode):

            state_for_model = ops.convert_to_tensor(state)
            state_for_model = ops.expand_dims(state_for_model, 0)

            # Predict action probabilities and estimated future rewards
            # from environment state
            action_probs, critic_value = model(state_for_model)
            critic_value_history.append(critic_value[0, 0])

            # Sample action from action probability distribution
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            action_probs_history.append(ops.log(action_probs[0, action]))

            # Apply the sampled action in our environment
            _new_state_raw, reward, terminated, truncated, _ = env.step(action) # Updated for gymnasium API
            done = terminated or truncated # 'done' in gym is 'terminated or truncated' in gymnasium

            # Apply the same reshaping logic to the new_state
            if not isinstance(_new_state_raw, np.ndarray):
                state = np.array([float(_new_state_raw)] + [0.0] * (num_inputs - 1), dtype=np.float32)
            else:
                if _new_state_raw.shape != (num_inputs,):
                    temp_state = np.zeros(num_inputs, dtype=np.float32)
                    temp_state[:min(_new_state_raw.shape[0], num_inputs)] = _new_state_raw[:min(_new_state_raw.shape[0], num_inputs)]
                    state = temp_state
                else:
                    state = _new_state_raw.astype(np.float32)

            rewards_history.append(reward)
            episode_reward += reward

            if done:
                break

        # Update running reward to check condition for solving
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # Calculate expected value from rewards
        # - At each timestep what was the total reward received after that timestep
        # - Rewards in the past are discounted by multiplying them with gamma
        # - These are the labels for our critic
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # Calculating loss values to update our network
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            # At this point in history, the critic estimated that we would get a
            # total reward = `value` in the future. We took an action with log probability
            # of `log_prob` and ended up receiving a total reward = `ret`.
            # The actor must be updated so that it predicts an action that leads to
            # high rewards (compared to critic's estimate) with high probability.
            diff = ret - value
            actor_losses.append(-log_prob * diff)  # actor loss

            # The critic must be updated so that it predicts a better estimate of
            # the future rewards.
            critic_losses.append(
                huber_loss(ops.expand_dims(value, 0), ops.expand_dims(ret, 0))
            )

        # Backpropagation
        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Clear the loss and reward history
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    # Log details
    episode_count += 1
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))

    if running_reward > 195:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

running reward: 10.71 at episode 10
running reward: 23.75 at episode 20
running reward: 23.21 at episode 30
running reward: 19.88 at episode 40
running reward: 18.74 at episode 50
running reward: 23.76 at episode 60
running reward: 41.83 at episode 70
running reward: 57.56 at episode 80
running reward: 62.48 at episode 90
running reward: 60.34 at episode 100
running reward: 49.85 at episode 110
running reward: 64.72 at episode 120
running reward: 56.03 at episode 130
running reward: 48.86 at episode 140
running reward: 43.36 at episode 150
running reward: 41.04 at episode 160
running reward: 46.97 at episode 170
running reward: 62.04 at episode 180
running reward: 77.40 at episode 190
running reward: 76.00 at episode 200
running reward: 83.99 at episode 210
running reward: 79.63 at episode 220
running reward: 76.83 at episode 230
running reward: 91.67 at episode 240
running reward: 94.76 at episode 250
running reward: 106.13 at episode 260
running reward: 122.25 at episode 270
running 

## Visualizations
In early stages of training:
![Imgur](https://i.imgur.com/5gCs5kH.gif)

In later stages of training:
![Imgur](https://i.imgur.com/5ziiZUD.gif)
